# Run (un)mitigated QV experiments

## Setup

In [ ]:
try:
    import mitiq
except ImportError:
    !pip install mitiq --quiet

try:
    import qiskit
except ImportError:
    !pip install qiskit --quiet

try:
    import qiskit_experiments
except ImportError:
    !pip uninstall matplotlib -y
    !pip install qiskit-experiments --quiet
    !pip install matplotlib~=3.4.0

In [ ]:
import functools
from typing import Iterable, List, Set

import numpy as np

import qiskit
import qiskit_experiments.library.quantum_volume as qv

from mitiq import zne

In [ ]:
try:
    _ = qiskit.IBMQ.load_account()
except Exception:
    print("IBMQ account not found.")

### Quantum volume functions

In [ ]:
def get_heavy_bitstrings(circuit: qiskit.QuantumCircuit) -> Set[str]:
    job = qiskit.execute(circuit, backend=qiskit.BasicAer.get_backend("statevector_simulator"))

    probs = list(job.result().get_counts().items())
    median = np.median([p for (_, p) in probs])
    return set(bitstring for (bitstring, p) in probs if p > median)


def estimate_heavy_output(
    circuits: List[qiskit.QuantumCircuit],
    backend: "qiskit.Sampler",
    qubits: List[int],
    nshots: int = 10_000,
    measure_all: bool = True,
    verbose: bool = False,
) -> List[float]:   
    circuits = [qiskit.transpile(circuit, basis_gates=["u1", "u2", "u3", "cx"], optimization_level=0) for circuit in circuits]

    # Determine the heavy bitstrings.
    heavy_bitstrings = [get_heavy_bitstrings(circuit) for circuit in circuits]

    if measure_all:
        for circuit in circuits:
            circuit.measure_all()

    # Run all circuits.
    job = qiskit.execute(
        circuits,
        backend=backend,
        initial_layout=qubits,
        shots=nshots,
        optimization_level=0,
    )

    # Count the number of heavy bitstrings sampled on the backend.
    heavy_counts = []
    for i in range(len(circuits)):
        heavy_counts.append(
            sum([job.result().get_counts(i).get(bitstring, 0) for bitstring in heavy_bitstrings[i]])
        )
    return heavy_counts

## Set experiment parameters

In [ ]:
# Notebook parameters.
MAX_BATCH_SIZE = 100
save_data = False

# Quantum volume parameters.
# backend = qiskit.IBMQ.get_provider(hub="your", group="credentials", project="here").get_backend("ibmq_quito")
#backend = qiskit.test.mock.FakeQuito()
backend = qiskit.providers.fake_provider.FakeQuito()
test_qubits = [
    [0, 1, 2],
    [0, 1, 3],
    [1, 3, 4],
    [0, 1, 2, 3],
    [0, 1, 3, 4],
    [0, 1, 2, 3, 4]
]
ntrials = 500
nshots = 10_000

# ZNE parameters.
scale_factors = [1, 3, 5, 7, 9]
fit = lambda s: zne.inference.RichardsonFactory(s)
scale_noise = functools.partial(
    zne.scaling.fold_gates_at_random,
    fidelities={"single": 1.0},
)

## Run experiment

In [ ]:
# Batching for raw experiment.
njobs = ntrials // MAX_BATCH_SIZE
step = MAX_BATCH_SIZE

# Batching for mitigated experiment.
njobs_zne = ntrials * len(scale_factors) // MAX_BATCH_SIZE
step_zne = MAX_BATCH_SIZE // len(scale_factors)
nshots_zne = nshots // len(scale_factors)

In [ ]:
all_raw_counts = []
all_scaled_counts = []

for qubits in test_qubits:
    print("Testing qubits", qubits)
    # Generate circuits.
    circuits = [
        qv.qv_experiment.QuantumVolumeCircuit(
            num_qubits=len(qubits),
            depth=len(qubits),
            seed=seed,
            classical_permutation=True,
        ) for seed in range(ntrials)
    ]
    circuits = [
        qiskit.transpile(circuit, basis_gates=["u1", "u2", "u3", "cx"], optimization_level=3)
        for circuit in circuits
    ]

    # Raw experiment.
    heavy_output_counts = []
    for i in range(njobs):
        to_run = circuits[i * step: (i + 1) * step]
        print("\r", end=f"Running QV batch {i + 1} / {njobs} with {len(to_run)} circuit(s).")
        heavy_output_counts.extend(
            estimate_heavy_output(to_run, backend=backend, qubits=qubits, nshots=nshots)
        )
    all_raw_counts.append(heavy_output_counts)
    print("\nMean heavy output probability:", sum(heavy_output_counts) / ntrials / nshots)

    # Mitigated experiment.
    heavy_output_counts_zne = []
    for i in range(njobs_zne):
        to_run = []
        for circuit in circuits[i * step_zne: (i + 1) * step_zne]:
            to_run.extend([scale_noise(circuit, scale_factor) for scale_factor in scale_factors])

        print("\r", end=f"Running QV batch {i + 1} / {njobs_zne} with {len(to_run)} circuit(s).")
        batched_counts = estimate_heavy_output(to_run, backend=backend, qubits=qubits, nshots=nshots_zne)

        for i in range(len(batched_counts) // len(scale_factors)):
            scaled_counts = batched_counts[i * len(scale_factors): (i + 1) * len(scale_factors)]
            all_scaled_counts.append(scaled_counts)
            zne_count = zne.inference.RichardsonFactory.extrapolate(scale_factors, scaled_counts)
            heavy_output_counts_zne.append(zne_count)

    print("\nMean mitigated heavy output probability:", sum(heavy_output_counts_zne) / ntrials / nshots_zne)

## Save data

In [ ]:
if save_data:
    np.savetxt(f"all_raw_counts_{backend.name()}.txt", all_raw_counts)
    np.savetxt(f"all_scaled_counts_{backend.name()}.txt", all_scaled_counts)